# A prototype workflow for geo-referenced burst (STAC)ks

In [1]:
%load_ext autoreload
%autoreload 2

import bursts
import requests
import json

In [2]:
asf_url = 'https://datapool.asf.alaska.edu/SLC/SB/S1B_IW_SLC__1SDV_20211111T002553_20211111T002623_029532_038657_AB46.zip'

In [3]:
auth = bursts.get_netrc_auth()
manifest, annotations = bursts.edl_download_metadata(asf_url, auth)

In [4]:
slc = bursts.SLCMetadata(asf_url, manifest, annotations)
swath = bursts.SwathMetadata(slc,'vv',swath_index=0)
burst = bursts.BurstMetadata(swath,burst_index=0)
item = burst.to_stac_item()
item.properties

{'stack_id': '227662_IW1',
 'wavelength': 0.05546576,
 'azimuth_steer_rate': 0.027757171601738514,
 'azimuth_time_interval': 0.002055556299999998,
 'slant_range_time': 0.005333045065152919,
 'starting_range': 799403.3443534819,
 'iw2_mid_range': 874424.562695774,
 'range_sampling_rate': 64345238.12571428,
 'range_pixel_spacing': 2.329562114715323,
 'azimuth_frame_rate': [[-2342.716413896163,
   444057.7084195721,
   -76224144.70273802],
  799403.3443534819,
  149896229.0],
 'doppler': [[22.80343, -4413.131, 6412146.0], 799782.3973890148, 149896229.0],
 'range_bandwidth': 56500000.0,
 'opera_id': 't106_227662_iw1',
 'center': (-92.26922797992722, -2.3030497747689522),
 'burst_index': 0,
 'first_valid_sample': 162,
 'last_valid_sample': 20589,
 'first_valid_line': 18,
 'last_valid_line': 1471,
 'range_window_type': 'hamming',
 'range_window_coefficient': 0.75,
 'rank': 9,
 'prf_raw_data': 1717.128973878037,
 'range_chirp_rate': 1078230321255.894,
 'sat:platform_international_designator':

In [5]:
item.validate()

['https://schemas.stacspec.org/v1.0.0/item-spec/json-schema/item.json',
 'https://stac-extensions.github.io/sat/v1.0.0/schema.json',
 'https://stac-extensions.github.io/sar/v1.0.0/schema.json']

In [6]:
catalog_url = 'https://cmr.uat.earthdata.nasa.gov/search/granules.umm_json?short_name=S1_SLC_BURSTS'
catalog = json.loads(requests.get(catalog_url).content)
len(catalog['items'])

10

In [7]:
ids = [x['meta']['native-id'] for x in catalog['items']]
cmr_urls = [f'{catalog_url}&granule_ur={x}' for x in ids]
cmr_urls

['https://cmr.uat.earthdata.nasa.gov/search/granules.umm_json?short_name=S1_SLC_BURSTS&granule_ur=S1_SLC_20211111T002553_VH_227662_IW1',
 'https://cmr.uat.earthdata.nasa.gov/search/granules.umm_json?short_name=S1_SLC_BURSTS&granule_ur=S1_SLC_20211111T002553_VV_227662_IW1',
 'https://cmr.uat.earthdata.nasa.gov/search/granules.umm_json?short_name=S1_SLC_BURSTS&granule_ur=S1_SLC_20211111T002554_VH_227662_IW2',
 'https://cmr.uat.earthdata.nasa.gov/search/granules.umm_json?short_name=S1_SLC_BURSTS&granule_ur=S1_SLC_20211111T002554_VV_227662_IW2',
 'https://cmr.uat.earthdata.nasa.gov/search/granules.umm_json?short_name=S1_SLC_BURSTS&granule_ur=S1_SLC_20211111T002555_VH_227663_IW3',
 'https://cmr.uat.earthdata.nasa.gov/search/granules.umm_json?short_name=S1_SLC_BURSTS&granule_ur=S1_SLC_20211111T002555_VV_227663_IW3',
 'https://cmr.uat.earthdata.nasa.gov/search/granules.umm_json?short_name=S1_SLC_BURSTS&granule_ur=S1_SLC_20211111T002556_VH_227663_IW1',
 'https://cmr.uat.earthdata.nasa.gov/sear

In [8]:
cmr_url = cmr_urls[0]

In [9]:
burst_instance = bursts.cmr_to_opera_burst(cmr_url,remote=True)

In [10]:
burst_instance

RemoteSentinel1BurstSLC(sensing_start=datetime.datetime(2021, 11, 11, 0, 25, 53, 695739), radar_center_frequency=5405000454.33435, wavelength=0.05546576, azimuth_steer_rate=0.027757171601738514, azimuth_time_interval=0.002055556299999998, slant_range_time=0.005333045065152919, starting_range=799403.3443534819, iw2_mid_range=874424.562695774, range_sampling_rate=64345238.12571428, range_pixel_spacing=2.329562114715323, shape=(1488, 20626), azimuth_fm_rate=Poly1d(order= 2, mean=799403, std=1.49896e+08), doppler=Doppler(poly1d=Poly1d(order= 2, mean=799782, std=1.49896e+08), lut2d=<isce3.ext.isce3.core.LUT2d object at 0x10f3e1770>), range_bandwidth=56500000.0, polarization='VH', burst_id='t106_227662_iw1', platform_id='S1B', center=<shapely.geometry.point.Point object at 0x10eea9d20>, border=[[-92.629432, -2.468856], [-92.234605, -2.385305], [-91.868591, -2.307914], [-91.911924, -2.104547], [-92.276925, -2.186245], [-92.670725, -2.274101], [-92.629432, -2.468856]], orbit=<isce3.ext.isce3.c

In [11]:
burst_instance.slc_to_file('./scratch/not_georeferenced.tif',fmt='GTiff')

In [12]:
!gdalinfo ./scratch/not_georeferenced.tif

Driver: GTiff/GeoTIFF
Files: ./scratch/not_georeferenced.tif
Size is 20626, 1488
Image Structure Metadata:
  INTERLEAVE=BAND
Corner Coordinates:
Upper Left  (    0.0,    0.0)
Lower Left  (    0.0, 1488.0)
Upper Right (20626.0,    0.0)
Lower Right (20626.0, 1488.0)
Center      (10313.0,  744.0)
Band 1 Block=20626x1 Type=CFloat32, ColorInterp=Gray


In [13]:
dem_path = './compass_outputs/input_data/dem_galapagos.tif'
output_dir = './scratch/geo_referenced'
scratch_dir = './scratch/tmp'
bursts.georeference_burst(burst_instance,dem_path,output_dir,scratch_dir)

data downloaded...
Chi squared: 0.000023
nbands: 1
geo-referenced!
nBlocks: 5
block: 0
-92.6933 , 0.0138889 , -2.20062, 0.0138889
journal: West limit may be insufficient for global height range
journal: East limit may be insufficient for global height range
journal: South limit may be insufficient for global height range
DEM interpolation Done
journal:
 -- Actual DEM bounds used:
 -- Top Left: -91.9226 0.0138889
 -- Bottom Right: -88.7699 -1.66944 
 -- Spacing: 0.000277778 -0.000277778
 -- Dimensions: 11351 6061
band: 0
get data block 
set output 
block: 1
-92.6932 , 0.0138889 , -2.2911, 0.0138889
journal: West limit may be insufficient for global height range
journal: East limit may be insufficient for global height range
journal: South limit may be insufficient for global height range
DEM interpolation Done
journal:
 -- Actual DEM bounds used:
 -- Top Left: -91.9226 0.0138889
 -- Bottom Right: -88.7699 -1.66944 
 -- Spacing: 0.000277778 -0.000277778
 -- Dimensions: 11351 6061
band: 0

'./scratch/geo_referenced/t106_227662_iw1_20211111_VH.tif'

In [14]:
!gdalinfo ./scratch/geo_referenced/t106_227662_iw1_20211111_VH.tif

Driver: GTiff/GeoTIFF
Files: ./scratch/geo_referenced/t106_227662_iw1_20211111_VH.tif
Size is 20323, 4142
Coordinate System is:
GEOGCRS["WGS 84",
    DATUM["World Geodetic System 1984",
        ELLIPSOID["WGS 84",6378137,298.257223563,
            LENGTHUNIT["metre",1]]],
    PRIMEM["Greenwich",0,
        ANGLEUNIT["degree",0.0174532925199433]],
    CS[ellipsoidal,2],
        AXIS["geodetic latitude (Lat)",north,
            ORDER[1],
            ANGLEUNIT["degree",0.0174532925199433]],
        AXIS["geodetic longitude (Lon)",east,
            ORDER[2],
            ANGLEUNIT["degree",0.0174532925199433]],
    ID["EPSG",4326]]
Data axis to CRS axis mapping: 2,1
Origin = (535653.834328493452631,9768295.004928255453706)
Pixel Size = (5.000000000000000,-10.000000000000000)
Metadata:
  AREA_OR_POINT=Area
Image Structure Metadata:
  INTERLEAVE=BAND
Corner Coordinates:
Upper Left  (  535653.834, 9768295.005) (Invalid angle,Invalid angle)
Lower Left  (  535653.834, 9726875.005) (Invalid angle,